In [93]:
pwd

'/home/wsuser/work'

In [97]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6M0x3NG5dGXOJK4jW2jsprymfUHFFTyizls8TOseoTYw',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'nutritionimageprocessingusingcnn-donotdelete-pr-oaym8kuyz8vgj7'
object_key = 'TRAIN_SET.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [98]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [99]:
ls

nutrition.h5  TRAIN_SET/


In [100]:
pwd

'/home/wsuser/work'

In [103]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of Layers
from tensorflow.keras import layers #A Layer consists of a tensor-in tensor-out computation function
#Dense Layer is the regular deeply connected neural network Layer
from tensorflow.keras.layers import Dense, Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Dropout #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator

In [104]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [105]:
x_train = train_datagen.flow_from_directory(
    r'/home/wsuser/work/TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse')
x_test = test_datagen.flow_from_directory(
    r'/home/wsuser/work/TRAIN_SET',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='sparse') 

Found 2626 images belonging to 5 classes.
Found 2626 images belonging to 5 classes.


In [106]:
print(x_train.class_indices)#checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [107]:
print(x_test.class_indices) #checking the number of classes

{'APPLES': 0, 'BANANA': 1, 'ORANGE': 2, 'PINEAPPLE': 3, 'WATERMELON': 4}


In [108]:
from collections import Counter as c 
c(x_train .labels)

Counter({0: 606, 1: 445, 2: 479, 3: 621, 4: 475})

In [109]:
model=Sequential()

In [110]:
# Initializing the CNN classifier = Sequential()
classifier = Sequential()
# First convolution layer and pooling
classifier.add(Conv2D(32,(3, 3), input_shape=(64, 64, 3),activation='relu')) 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer 
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Flattening the Layers
classifier.add(Flatten())
# Adding fully connected Layer a
classifier.add(Dense (units=128, activation='relu'))
classifier.add(Dense (units=5, activation='softmax')) # softmax for more than 2

In [111]:
classifier.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 6272)              0         
                                                                 
 dense_4 (Dense)             (None, 128)              

In [112]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [113]:
##Fitting the model
classifier.fit_generator(
    generator=x_train,steps_per_epoch = len(x_train), 
    epochs=20,validation_data=x_test,validation_steps = len(x_test)) # No of images in test set


/tmp/wsuser/ipykernel_164/2706448856.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/20
526/526 [==============================] - 35s 65ms/step - loss: 0.1845 - accuracy: 0.9288 - val_loss: 7.0065e-04 - val_accuracy: 1.0000
Epoch 2/20
526/526 [==============================] - 36s 68ms/step - loss: 0.0397 - accuracy: 0.9886 - val_loss: 4.5378e-04 - val_accuracy: 1.0000
Epoch 3/20
526/526 [==============================] - 33s 63ms/step - loss: 0.0016 - accuracy: 0.9992 - val_loss: 7.3947e-05 - val_accuracy: 1.0000
Epoch 4/20
526/526 [==============================] - 36s 69ms/step - loss: 1.4494e-04 - accuracy: 1.0000 - val_loss: 2.1751e-05 - val_accuracy: 1.0000
Epoch 5/20
526/526 [==============================] - 34s 64ms/step - loss: 4.5869e-05 - accuracy: 1.0000 - val_loss: 1.4541e-05 - val_accuracy: 1.0000
Epoch 6/20
526/526 [==============================] - 35s 66ms/step - loss: 2.4958e-05 - accuracy: 1.0000 - val_loss: 1.0741e-05 - val_accuracy: 1.0000
Epoch 7/20
526/526 [==============================] - 34s 64ms/step - loss: 4.9034e-05 - accuracy: 1

In [114]:
ls

nutrition.h5  TRAIN_SET/


In [115]:
classifier.save('nutrition.h5')

In [116]:
ls

nutrition.h5  TRAIN_SET/


In [117]:
pwd

'/home/wsuser/work'

In [118]:
### Predicting our results
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("nutrition.h5") #Loading the model for testing
from tensorflow.keras.preprocessing import image

In [119]:
img =image.load_img(r"/home/wsuser/work/TRAIN_SET/ORANGE/100_100.jpg",grayscale=False, target_size= (64,64))
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis=0) #changing the shape =
y =np.argmax(model.predict(x),axis=1)
index=['APPLES', 'BANANA', 'ORANGE','PINEAPPLE','WATERMELON']
index[y[0]]

'ORANGE'

# IBM Deployment

In [120]:
!pip install watson-machine-learning-client

In [121]:
!pip install keras==2.2.4
!pip install tensorflow==2.5.0

In [122]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"BGB_wP_pV2ZAoyBd0J0tJ8SYUf1XpCvuzPwkefw4XhCn"
}

client=APIClient(wml_credentials)

In [123]:
client

In [130]:
def guid_space_name(client,nutrition_deploy):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==nutrition_deploy)['metadata']['id'])

# Deployment

In [131]:
space_uid=guid_space_name(client,'nutrition_deploy')
print("Space UID " + space_uid)

Space UID 94886823-18d6-449e-b756-0770f08bfa66


In [132]:
client.set.default_space(space_uid)

'SUCCESS'

In [133]:
client.software_specifications.list(200)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

In [136]:
software_space_uid=client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')

In [137]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [141]:
!tar -zcvf nutrition-classification-model.tgz nutrition.h5

nutrition.h5


In [143]:
model_details=client.repository.store_model(model ='nutrition-classification-model.tgz', meta_props={
    client.repository.ModelMetaNames.NAME: "CNN Model Building",
    client.repository.ModelMetaNames.TYPE: 'tensorflow_2.7',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_space_uid
})

In [144]:
model_details=client.repository.get_model_id(model_details)

In [152]:
model_id

'22066c6c-17f6-4696-b7db-0301b44f0750'

In [154]:
ls

nutrition-classification-model.tgz  nutrition.h5  TRAIN_SET/


In [156]:
client.repository.download(model_id,'nutrition.tar.gb')

Successfully saved model content to file: 'nutrition.tar.gb'


'/home/wsuser/work/nutrition.tar.gb'

In [157]:
pwd

'/home/wsuser/work'